# Doubly Robust Estimation

This notebook implements doubly robust estimation:
- Propensity score modeling
- Outcome regression modeling
- AIPW estimation
- Confidence intervals and significance testing

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error
from src.models.doubly_robust import doubly_robust_estimation, aipw_estimation
from src.evaluation.causal_metrics import compute_ate, lift_calculation
from scipy import stats

# Load preprocessed data
df = pd.read_csv('../data/processed/preprocessed_ad_data.csv')
confounders = pd.read_csv('../data/processed/confounders.csv')['confounder'].tolist()

print(f"Loaded {len(df)} observations with {len(confounders)} confounders")
print(f"Treatment distribution: {df['treatment'].value_counts().to_dict()}")

# Calculate naive estimate for comparison
naive_ate = df[df['treatment']==1]['conversion'].mean() - df[df['treatment']==0]['conversion'].mean()
true_ate = df['true_effect'].iloc[0]
print(f"Naive ATE: {naive_ate:.4f}")
print(f"True ATE: {true_ate:.4f}")

## Step 1: Doubly Robust Estimation

In [ ]:
# Run doubly robust estimation
dr_results = doubly_robust_estimation(df, confounders)

print("=== DOUBLY ROBUST ESTIMATION RESULTS ===")
print(f"Estimated ATE: {dr_results['ate']:.4f}")
print(f"Standard Error: {dr_results['ate_se']:.4f}")
print(f"95% Confidence Interval: [{dr_results['ate_ci_lower']:.4f}, {dr_results['ate_ci_upper']:.4f}]")
print(f"")
print(f"Components:")
print(f"  Direct Effect: {dr_results['direct_effect']:.4f}")
print(f"  Treated Residual: {dr_results['treated_residual']:.4f}")
print(f"  Control Residual: {dr_results['control_residual']:.4f}")

# Statistical significance
t_stat = dr_results['ate'] / dr_results['ate_se']
p_value = 2 * (1 - stats.norm.cdf(abs(t_stat)))
print(f"\nStatistical Significance:")
print(f"  t-statistic: {t_stat:.3f}")
print(f"  p-value: {p_value:.6f}")
print(f"  Significant at 5%: {'Yes' if p_value < 0.05 else 'No'}")

# Bias correction
bias_correction = naive_ate - dr_results['ate']
bias_pct = (bias_correction / naive_ate) * 100 if naive_ate != 0 else 0
print(f"\nBias Correction:")
print(f"  Bias magnitude: {bias_correction:.4f}")
print(f"  Bias percentage: {bias_pct:.1f}%")
print(f"  Direction: {'Overstatement' if bias_correction > 0 else 'Understatement'}")

## Step 2: Model Validation

In [ ]:
# Validate component models
X = df[confounders]
T = df['treatment']
Y = df['conversion']

# Validate propensity model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

propensity_model = LogisticRegression(random_state=42, max_iter=1000)
ps_scores = cross_val_score(propensity_model, X, T, cv=5, scoring='roc_auc')

print("=== MODEL VALIDATION ===")
print(f"Propensity Model Performance:")
print(f"  Cross-val AUC: {ps_scores.mean():.3f} ± {ps_scores.std():.3f}")
print(f"  Individual folds: {[f'{score:.3f}' for score in ps_scores]}")

# Validate outcome models
outcome_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Control group outcome model
control_indices = T == 0
control_scores = cross_val_score(outcome_model, X[control_indices], Y[control_indices], cv=5, scoring='neg_mean_squared_error')

# Treatment group outcome model  
treated_indices = T == 1
treated_scores = cross_val_score(outcome_model, X[treated_indices], Y[treated_indices], cv=5, scoring='neg_mean_squared_error')

print(f"\nOutcome Model Performance:")
print(f"  Control model RMSE: {np.sqrt(-control_scores.mean()):.4f} ± {np.sqrt(control_scores.std()):.4f}")
print(f"  Treatment model RMSE: {np.sqrt(-treated_scores.mean()):.4f} ± {np.sqrt(treated_scores.std()):.4f}")

# Check model assumptions
print(f"\n=== MODEL ASSUMPTIONS ===")
print(f"Propensity scores range: [{dr_results['propensity_scores'].min():.3f}, {dr_results['propensity_scores'].max():.3f}]")
print(f"Extreme propensity scores (<0.1 or >0.9): {((dr_results['propensity_scores'] < 0.1) | (dr_results['propensity_scores'] > 0.9)).sum()}")
print(f"Overlap condition: {'✅ Good' if dr_results['propensity_scores'].min() > 0.01 and dr_results['propensity_scores'].max() < 0.99 else '⚠️ Concern'}")

## Step 3: Diagnostic Plots

In [ ]:
# Create diagnostic visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Doubly Robust Estimation Diagnostics', fontsize=16)

# 1. Propensity score distribution
treated_ps = dr_results['propensity_scores'][T == 1]
control_ps = dr_results['propensity_scores'][T == 0]

axes[0,0].hist(control_ps, bins=30, alpha=0.7, label='Control', color='blue', density=True)
axes[0,0].hist(treated_ps, bins=30, alpha=0.7, label='Treated', color='red', density=True)
axes[0,0].set_xlabel('Propensity Score')
axes[0,0].set_ylabel('Density')
axes[0,0].set_title('Propensity Score Distribution')
axes[0,0].legend()

# 2. Predicted outcomes vs actual
axes[0,1].scatter(dr_results['mu0'], Y, alpha=0.6, label='μ₀(X)', color='blue')
axes[0,1].scatter(dr_results['mu1'], Y, alpha=0.6, label='μ₁(X)', color='red')
axes[0,1].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[0,1].set_xlabel('Predicted Outcome')
axes[0,1].set_ylabel('Actual Outcome')
axes[0,1].set_title('Outcome Model Predictions')
axes[0,1].legend()

# 3. Residuals analysis
treated_residuals = Y[T == 1] - dr_results['mu1'][T == 1]
control_residuals = Y[T == 0] - dr_results['mu0'][T == 0]

axes[0,2].scatter(dr_results['mu1'][T == 1], treated_residuals, alpha=0.6, color='red', label='Treated')
axes[0,2].scatter(dr_results['mu0'][T == 0], control_residuals, alpha=0.6, color='blue', label='Control')
axes[0,2].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[0,2].set_xlabel('Predicted Outcome')
axes[0,2].set_ylabel('Residuals')
axes[0,2].set_title('Residual Analysis')
axes[0,2].legend()

# 4. Treatment effect by propensity quintiles
ps_quintiles = pd.qcut(dr_results['propensity_scores'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
quintile_effects = []
for q in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']:
    mask = ps_quintiles == q
    treated_mean = Y[(T == 1) & mask].mean() if ((T == 1) & mask).sum() > 0 else 0
    control_mean = Y[(T == 0) & mask].mean() if ((T == 0) & mask).sum() > 0 else 0
    quintile_effects.append(treated_mean - control_mean)

axes[1,0].bar(['Q1', 'Q2', 'Q3', 'Q4', 'Q5'], quintile_effects, alpha=0.8, color='green')
axes[1,0].axhline(y=dr_results['ate'], color='red', linestyle='--', label=f"Overall ATE = {dr_results['ate']:.3f}")
axes[1,0].set_xlabel('Propensity Score Quintile')
axes[1,0].set_ylabel('Treatment Effect')
axes[1,0].set_title('Treatment Effect by Propensity Quintile')
axes[1,0].legend()

# 5. Confidence interval visualization
methods = ['Naive', 'Doubly Robust', 'True Effect']
estimates = [naive_ate, dr_results['ate'], true_ate]
errors = [0, dr_results['ate_se'], 0]
colors = ['red', 'blue', 'green']

bars = axes[1,1].bar(methods, estimates, yerr=[0, errors[1], 0], capsize=5, color=colors, alpha=0.7)
axes[1,1].set_ylabel('Treatment Effect')
axes[1,1].set_title('Method Comparison')

# Add value labels
for bar, est in zip(bars, estimates):
    axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                  f'{est:.4f}', ha='center', va='bottom')

# 6. Bootstrap confidence interval
n_bootstrap = 1000
bootstrap_ates = []

np.random.seed(42)
for _ in range(n_bootstrap):
    # Bootstrap sample
    idx = np.random.choice(len(df), size=len(df), replace=True)
    df_boot = df.iloc[idx]
    
    try:
        dr_boot = doubly_robust_estimation(df_boot, confounders)
        bootstrap_ates.append(dr_boot['ate'])
    except:
        continue

if bootstrap_ates:
    axes[1,2].hist(bootstrap_ates, bins=30, alpha=0.7, color='purple', density=True)
    axes[1,2].axvline(dr_results['ate'], color='red', linestyle='-', linewidth=2, label='Point Estimate')
    axes[1,2].axvline(np.percentile(bootstrap_ates, 2.5), color='red', linestyle='--', alpha=0.7, label='95% CI')
    axes[1,2].axvline(np.percentile(bootstrap_ates, 97.5), color='red', linestyle='--', alpha=0.7)
    axes[1,2].set_xlabel('Bootstrap ATE Estimates')
    axes[1,2].set_ylabel('Density')
    axes[1,2].set_title('Bootstrap Distribution')
    axes[1,2].legend()

plt.tight_layout()
plt.show()

## Step 4: Business Impact Analysis

In [ ]:
# Calculate business metrics
treated_outcomes = df[df['treatment']==1]['conversion']
control_outcomes = df[df['treatment']==0]['conversion']

lift_metrics = lift_calculation(treated_outcomes, control_outcomes)
ate_metrics = compute_ate(treated_outcomes, control_outcomes)

print("=== BUSINESS IMPACT ANALYSIS ===")
print(f"\n📊 Conversion Rates:")
print(f"  Control Group: {lift_metrics['control_rate']:.3f} ({lift_metrics['control_rate']*100:.1f}%)")
print(f"  Treatment Group: {lift_metrics['treated_rate']:.3f} ({lift_metrics['treated_rate']*100:.1f}%)")

print(f"\n📈 Lift Metrics:")
print(f"  Absolute Lift (DR): {dr_results['ate']:.4f}")
print(f"  Relative Lift (DR): {(dr_results['ate'] / lift_metrics['control_rate'] * 100):.1f}%")
print(f"  Incremental per 1000: {dr_results['ate'] * 1000:.1f} conversions")

print(f"\n💰 Financial Impact (Example):")
monthly_impressions = 1000000
aov = 50  # Average order value
incremental_conversions = dr_results['ate'] * monthly_impressions
additional_revenue = incremental_conversions * aov

print(f"  Monthly Impressions: {monthly_impressions:,}")
print(f"  Incremental Conversions: {incremental_conversions:,.0f}")
print(f"  Additional Revenue: ${additional_revenue:,.0f} (assuming ${aov} AOV)")

print(f"\n🎯 Confidence & Significance:")
print(f"  Statistical Significance: {'✅ Yes' if p_value < 0.05 else '❌ No'} (p = {p_value:.4f})")
print(f"  95% Confidence Interval: [{dr_results['ate_ci_lower']:.4f}, {dr_results['ate_ci_upper']:.4f}]")
print(f"  Effect Size: {'Large' if abs(dr_results['ate']) > 0.01 else 'Medium' if abs(dr_results['ate']) > 0.005 else 'Small'}")

# Save results
results_summary = {
    'method': 'doubly_robust',
    'ate': dr_results['ate'],
    'ate_se': dr_results['ate_se'],
    'ate_ci_lower': dr_results['ate_ci_lower'],
    'ate_ci_upper': dr_results['ate_ci_upper'],
    'p_value': p_value,
    'bias_correction': bias_correction,
    'bias_correction_pct': bias_pct
}

import pickle
with open('../results/doubly_robust_results.pkl', 'wb') as f:
    pickle.dump(dr_results, f)

print("\n✅ Results saved to '../results/doubly_robust_results.pkl'")